In [1]:
import pandas as pd
import numpy as np
import json
import seaborn as sns
import os
import cv2
from matplotlib import pyplot as plt

### train.json bbox 40이상 있는 image 제거

In [55]:
data_json = '../dataset/train.json'
data_json

'../dataset/train.json'

In [114]:
with open(data_json, 'r') as outfile:
    data = (json.load(outfile))

df_image = pd.DataFrame(data['images']) #이미지 데이터
df_annotation = pd.DataFrame(data['annotations']) #bbox 
df_categories = pd.DataFrame(data['categories'])#categories

In [116]:
df_image.tail()

,width,height,file_name,license,flickr_url,coco_url,date_captured,id
4878,1024,1024,train/4878.jpg,0,None,None,2021-01-04 14:36:54,4878
4879,1024,1024,train/4879.jpg,0,None,None,2021-04-27 13:46:18,4879
4880,1024,1024,train/4880.jpg,0,None,None,2020-12-24 11:25:46,4880
4881,1024,1024,train/4881.jpg,0,None,None,2021-01-05 11:01:18,4881
4882,1024,1024,train/4882.jpg,0,None,None,2020-12-23 16:20:30,4882


In [57]:
df_count = df_annotation['image_id'].value_counts().to_frame().rename(columns={'image_id':'counts'}).reset_index()
df_img_over_40 = df_count.loc[df_count['counts'] > 30]
df_o4_list = list(df_img_over_30['index']) #bbox 40이상 image list

### annotation bbox 크기 2000이하 제거 

In [118]:
df_area_u_2000 = df_annotation.loc[df_annotation['area'] <= 2000]
df_area_list = list(df_area_u_2000['id'])

In [120]:
images_list = []
for obj in data['images']:
    
    if obj['id'] not in df_o4_list:
        images_list.append(obj)
len(images_list)

4831

In [112]:
annotations_list = []
for obj in data['annotations']:
    
    if obj['image_id'] not in df_o4_list and obj['id'] not in df_area_list:
        annotations_list.append(obj)

len(annotations_list)

20086

## 새로운 json 저장

In [113]:
new_json = json.dumps({'images':images_list, 'annotations':annotations_list, 'categories':data['categories']})
parsed_json = json.loads(new_json)

file_path = '/opt/ml/detection/dataset/refined_train.json'

with open(file_path, 'w') as outfile:
    json.dump(parsed_json, outfile, indent=4)